In [1]:
# !pip install transformers

In [2]:
import transformers
import pandas as pd
import os
from collections import defaultdict
import re

/Users/ludsil/miniconda3/envs/prosam/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# df = pd.read_csv('./data/esg_reports/ABT_Abbott/ABT_2021.txt')

In [4]:
reports = {}

for folder in os.listdir('./data/esg_reports'):
    for file in os.listdir(f'./data/esg_reports/{folder}'):
        if file.endswith('2021.txt'):
            df = pd.read_csv(f'./data/esg_reports/{folder}/{file}')
            # pick out ticker
            ticker = folder.split('_')[0]
            reports[ticker] = df

In [5]:
# df_abt = reports['ABT']
# df_abt.head()

In [9]:
# Topics as keys, and a list of all sentences classified as that topic as values

topics_by_company = defaultdict(lambda: defaultdict(lambda: []))

for company in reports:

    sentences_by_topic = defaultdict(lambda: [])

    for index, row in reports[company].iterrows():
        if row['ESG BERT Topic 1'][0] != 'NON-ESG':
            input_string = row['ESG BERT Topic 1']
            match = re.match(r"\('(.+)',\s*(\d+\.\d+)\)", input_string)
            topic = match.group(1)
            number = float(match.group(2))
            sentences_by_topic[topic].append((row['Sentence'], number))
    topics_by_company[company] = sentences_by_topic

# Funkar

dict_keys(['BSX', 'BDX', 'EW', 'BAX', 'ISRG', 'MDT', 'DXCM', 'PODD', 'SYK', 'HOLX', 'RMD', 'ABT', 'COO', 'IDXX', 'TFX', 'XRAY', 'ZBH'])

In [11]:
# Sort sentences by topic by the number
for company in topics_by_company:
    for topic in topics_by_company[company]:
        sentences_by_topic = topics_by_company[company]
        sentences_by_topic[topic] = sorted(sentences_by_topic[topic], key=lambda x: x[1], reverse=True)

# Funkar

In [7]:
# sentence_list = []
# # Add the 10 first sentences of 'Access_And_Affordability' to the document
# for sentence, number in sentences_by_topic['Access_And_Affordability'][:10]:
#     sentence_list.append(sentence)

# document = " ".join(sentence_list)

In [8]:
# summarizer = transformers.pipeline('summarization')
# summary = summarizer(document, max_length=100, min_length=10, do_sample=False)
# print(summary[0]['summary_text'])

In [13]:
# For some reason still uses abbott first?

for company in topics_by_company:
    print(company)
    for topic in topics_by_company[company]:
        sentence_list = []
        sentences_by_topic = topics_by_company[company]
        for sentence, number in sentences_by_topic[topic][:5]:
            # print(topic, sentence, number)
            sentence_list.append(sentence)

        document = " ".join(sentence_list)

        summarizer = transformers.pipeline('summarization')
        summary = summarizer(document, max_length=100, min_length=10, do_sample=False)
        print(topic, ": " , summary[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


BSX


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Business_Model_Resilience :   2021 Performance Report: Transforming care Investing in   our people Accelerating   possibilities Protecting the  environment Creating value  responsibly . Good governance  reflects our values  Compliance, ethics  and integrity Protecting human rights  across our company  and supply chain Keeping our supply chain  resilient and reliable Risk management  and global security Keeping our products   and patient data secure .


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


...

Employee_Health_And_Safety :   For 2021, Boston Scientific expanded the relevant headcount in scope for TRIR, TRIFR, LTIFR  and OLTR reporting to include 98% of our population . 2021 Performance Report: Transforming care Investing in our people Accelerating   possibilities Protecting the  environment Creating value  responsibly .


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Employee_Engagement_Inclusion_And_Diversity :   Boston Scientific increased representation of women in supervisor and manager roles globally to 41.1% and multicultural employees in the U.S. and Puerto Rico to 21.6% In 2021, we made progress toward our goals by taking the following actions .


Token indices sequence length is longer than the specified maximum sequence length for this model (2277 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self